In [24]:
from func.datePoint import datePoint
from func.util import getProfile, getDateList, getDateInput, readGridDataFromFile
from func.valueMaps import ValueMap

from datetime import datetime
import json
import matplotlib.pyplot as plt
import time
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import math


In [7]:
%matplotlib qt 

In [3]:
profileName = "SRRPeriod"

with open("./config/download.json", "r", encoding="utf-8") as dataProfileFile:
    configData = json.load(dataProfileFile)

configData = configData[profileName]
dateList = getDateList(datetime.strptime(configData["startDate"], "%d/%m/%Y"), datetime.strptime(configData["endDate"], "%d/%m/%Y"))

for dateIndex, date in enumerate(dateList):
    dateList[dateIndex] = datePoint(date)

dataTypeCount = len(configData["dataTypes"])

In [4]:
startTime = time.time()
# Load data about the energy grid
print("Loading Grid Data")
if "grid" in configData["dataTypes"]:
    dataTypeCount = dataTypeCount - 1
    with open(f"./data/processed/grid/{profileName}.csv", "r") as gridDataFile:
        gridLines = gridDataFile.readlines()
        gridLabels = gridLines[0].split(",")[1:]
        

        for date in dateList:
            for rowIndex, row in enumerate(gridLines):
                if row.split(",")[0] == date.getDateStr():
                    dataArr = {}
                    for valIndex, val in enumerate(gridLabels):
                        dataArr[val] = float(row.split(",")[valIndex+1])
                    date.energyData = dataArr
                    gridLines.remove(row)
                    break
            # print(f"Loading Grid Data {str(round((rowIndex+1)/len(gridLines), 2) * 100)[:4]}%", end="\r", flush=True)
# Load Climate Data
print("Loading Climate Data")
if dataTypeCount > 0:
    with open(f"./data/processed/climate/{profileName}.csv", "r") as climateFile:
        climateData = climateFile.readlines()
        climateLabels = climateData[0].split(",")
        
        # Automatically determine the column each datatype is in
        dataIndexes = {}
        for dataType in configData["dataTypes"]:
            for labelIndex, label in enumerate(climateLabels):
                if dataType in label:
                    dataIndexes[dataType] = labelIndex
                    break
        
        # Validate that start and end dates are the same
        if not (climateData[1].split(",")[0] == dateList[0].getDateStr() and climateData[-1].split(",")[0] == dateList[-1].getDateStr()):
            print(f"FATAL ERROR\nThe Climate data and studied period do not have the same date range")
            print(f"Study Period date Range: {dateList[0].getDateStr()} - {dateList[-1].getDateStr()}")
            print(f"Climate Data date Range: {climateData[1].split(',')[0]} - {climateData[-1].split(',')[0]}")
            exit()

        # Load data
        climateData = climateData[1:] # Remove labels
        for rowIndex, row in enumerate(climateData):
            if row.split(",")[0] == dateList[rowIndex].getDateStr():
                for dataType in configData["dataTypes"]:
                    if dataType != "grid":
                        exec(f"dateList[{rowIndex}].{dataType} = float(row.split(',')[dataIndexes[dataType]])")

# Load Wind Data
print("Loading Wind Data")
if configData["windFile"] != "":
    with open(f"./data/processed/wind/{configData['windFile']}", "r") as windFile:
        windData = windFile.readlines()

        # Validate that start and end dates are the same
        if not(windData[1].split(",")[0] == dateList[0].getDateStr() and windData[-1].split(",")[0] == dateList[-1].getDateStr()):
            print(f"FATAL ERROR\nThe Wind data and studied period do not have the same date range")
            print(f"Study Period date Range: {dateList[0].getDateStr()} - {dateList[-1].getDateStr()}")
            print(f"Wind Data date Range: {windData[1].split(',')[0]} - {windData[-1].split(',')[0]}")
            exit()

        windData = windData[1:]
        for rowIndex, row in enumerate(windData):
            row = row.split(",")
            if row[0] == dateList[rowIndex].getDateStr():
                dateList[rowIndex].windspeed = float(row[1])
                dateList[rowIndex].windangle = float(row[4])

print(f"Loaded in {round(time.time() - startTime,3)} seconds")

Loading Grid Data
Loading Climate Data
Loading Wind Data
Loaded in 0.291 seconds


In [203]:
# Create Model to predict energy demand
testValCount = 200 # The number of data points that are EXCLUDED and used to assess the model
inputData = np.empty((len(dateList), 6))
for dateIndex, date in enumerate(dateList):
    inputData[dateIndex][0] = date.dayoftheyear
    inputData[dateIndex][1] = date.temperature
    inputData[dateIndex][2] = date.irradiance
    inputData[dateIndex][3] = date.pressure
    inputData[dateIndex][4] = date.rainfall
    inputData[dateIndex][5] = date.energyData["au.nem.nsw1.demand.energy (GWh)"]

np.random.shuffle(inputData)
trainingData, testData = inputData[:len(dateList)-testValCount,:], inputData[len(dateList)-testValCount:,:]

In [204]:
def predict(xValue, scalerX, scalerY, svr):
    transformedVal = scalerX.transform(xValue)
    predictedVal = svr.predict(transformedVal)
    val = scalerY.inverse_transform(predictedVal)
    return val
#     return scalerY.inverse_transform(svr.predict(scalerX.transform(xValue)))

In [205]:
scalerX = StandardScaler()
scalerY = StandardScaler()
X = scalerX.fit_transform(trainingData[:, :5])
Y = scalerY.fit_transform(trainingData[:,5].reshape(-1, 1))

In [206]:
# create an SVR model with a linear kernel 
demandSVR = SVR(kernel='rbf') 
  
# train the model on the data 
demandSVR.fit(X, Y.ravel()) 

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [207]:
val = predict(
    testData[:, :5],
    scalerX,
    scalerY,
    demandSVR
)

In [210]:
plt.scatter(testData[:,1], val, label="Predicted")
plt.scatter(testData[:,1], testData[:,5], label="Observed")
plt.legend()
plt.show()